In [1]:
import pandas
from geosnap import Community
from geosnap import datasets
import fiona
import geopandas
%matplotlib inline
import matplotlib.pyplot as plt
import segregation


## 5/20 need to make cds in LCFF same length as CDS in performance (ela) dataset, with 6 zeroes. Add trailing zeroes
## also on performance(ela) dataset, need to add single leading zero for single digit county portion of cds code.

C:\Users\fiend\.conda\envs\capstone\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.cluster.spectral module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.cluster. Anything that cannot be imported from sklearn.cluster is now part of the private API.
  warnings.warn(message, FutureWarning)


TO DO: <br>
- What are the 9 extra entries in the school district shapefiles?<br><br>
- Construct community from school district files/merge census data<br><br>
- Figure out discrepancy between n of districts and LCFF data <br><br>
- Create dictionaries using achievement data files in a *student group > ELA > math* format <br><br>
- Determine viability of FRPM as an indicator of segregation vs traditional segregation measures


In [59]:
pandas.set_option('display.max_rows', 500)
pandas.set_option('display.max_column', 500)

In [2]:
ele = geopandas.read_file("districts_tiger/tl_2015_us_sde.shp")
# Reading in shapefiles from census/tiger

In [3]:
seco = geopandas.read_file("districts_tiger/tl_2015_us_sds.shp")

In [4]:
uni = geopandas.read_file("districts_tiger/tl_2015_us_sdu.shp")

In [8]:
ele = ele.loc[ele['STATEFP'] == '06']
# slicing out California
# elementary school districts in California

In [9]:
seco = seco.loc[seco['STATEFP'] == '06']
# secondary school districts in California

In [10]:
uni = uni.loc[uni['STATEFP'] == '06']
# unified school districts in California

In [18]:
len(ele)+len(seco)+len(uni)
#

986

## Combining shapefiles into single geodataframe

In [19]:
ele['origin'] = 'elementary'

In [20]:
seco['origin'] = 'secondary'

In [21]:
uni['origin'] = 'unified'

In [23]:
adistricts = uni.append([ele, seco])
#all districts

In [24]:
adistricts = adistricts.sort_values('NAME', ascending=True)

In [25]:
adistricts.head()

,STATEFP,UNSDLEA,GEOID,NAME,LSAD,LOGRADE,HIGRADE,MTFCC,SDTYP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,origin,ELSDLEA,SCSDLEA
645,06,01620,0601620,ABC Unified School District,00,KG,12,G5420,None,E,38772821,556042,+33.8637711,-118.0665456,"POLYGON ((-118.10857 33.88775, -118.10843 33.8...",unified,NaN,NaN
29,06,NaN,0601650,Acalanes Union High School District,00,09,12,G5410,None,E,205137702,4413448,+37.8772150,-122.1288576,"POLYGON ((-122.26126 37.92143, -122.26009 37.9...",secondary,NaN,01650
403,06,NaN,0601680,Ackerman Elementary School District,00,KG,08,G5400,None,E,11308882,7628,+38.9366318,-121.0507823,"POLYGON ((-121.07045 38.92923, -121.07045 38.9...",elementary,01680,NaN
658,06,00001,0600001,Acton-Agua Dulce Unified School District,00,KG,12,G5420,None,E,555175128,182565,+34.4509057,-118.2276141,"POLYGON ((-118.38010 34.50369, -118.37724 34.5...",unified,NaN,NaN
376,06,NaN,0601710,Adelanto Elementary School District,00,KG,06,G5400,None,E,868470886,832474,+34.6306380,-117.5205628,"POLYGON ((-117.66785 34.79074, -117.66784 34.7...",elementary,01710,NaN


In [26]:
adistricts.shape

(986, 18)

### Harmonizing data

In [27]:
adsgeo = adistricts['NAME']
# all districs shapfile geodataframe

In [44]:
adsgeo.unique().shape
# todo: some districts listed twice

(970,)

In [54]:
dupes = adistricts[adistricts['NAME'].isin(adistricts['NAME'].value_counts()[adistricts['NAME'].value_counts()>2].index)]
print(dupes['NAME'].unique())
# 3 Jefferson
# 3 Lakeside
# 3 Pioneer

['Jefferson Elementary School District'
 'Lakeside Union Elementary School District'
 'Pioneer Union Elementary School District']


In [ ]:
# repeat for perfomance data
# loop over both sets of names to check which one's match <-- YOU ARE HERE
# reverse loop to re-verify, see which ones are missing
# 2 lists of 
# create list of problem children

In [55]:
a1 = adsgeo.astype(str)

## ELA acheivement indicator data
Find the codebook [here](https://www.cde.ca.gov/ta/ac/cm/ela18.asp)

In [251]:
ela = pandas.read_csv('lcff_data/eladownload2018.csv')
ela.shape
#Current Status - Average distance from Standard of students who took the Smarter Balanced summative assessment in ELA
#can use with 'prior status' or 'change' to illustrate performance

C:\Users\fiend\.conda\envs\capstone\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (6,23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(148838, 32)

In [252]:
ela.columns

Index(['cds', 'rtype', 'schoolname', 'districtname', 'countyname',
       'charter_flag', 'coe_flag', 'dass_flag', 'studentgroup', 'currdenom',
       'currdenom_swd', 'currstatus', 'priordenom', 'priordenom_swd',
       'priorstatus', 'change', 'statuslevel', 'changelevel', 'color', 'box',
       'hscutpoints', 'curradjustment', 'prioradjustment', 'pairshare_method',
       'caa_denom', 'caa_level1_num', 'caa_level1_pct', 'caa_level2_num',
       'caa_level2_pct', 'caa_level3_num', 'caa_level3_pct', 'ReportingYear'],
      dtype='object')

In [253]:
ela = ela.loc[ela['rtype'] == 'D']
# select district level performance
len(ela.districtname.unique())

969

In [232]:
ela['cds'].astype(str)
elat = set(ela['cds'])

,cds,rtype,schoolname,districtname,countyname,charter_flag,coe_flag,dass_flag,studentgroup,currdenom,currdenom_swd,currstatus,priordenom,priordenom_swd,priorstatus,change,statuslevel,changelevel,color,box,hscutpoints,curradjustment,prioradjustment,pairshare_method,caa_denom,caa_level1_num,caa_level1_pct,caa_level2_num,caa_level2_pct,caa_level3_num,caa_level3_pct,ReportingYear
17,1100170000000,D,NaN,Alameda County Office of Education,Alameda,NaN,Y,NaN,AA,2,NaN,NaN,9.0,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018
18,1100170000000,D,NaN,Alameda County Office of Education,Alameda,NaN,Y,NaN,ALL,20,NaN,-157.0,17.0,NaN,-186.0,29.0,1,5,0,0,NaN,-12.75,-16.5,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2018
19,1100170000000,D,NaN,Alameda County Office of Education,Alameda,NaN,Y,NaN,AS,1,NaN,NaN,0.0,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018
20,1100170000000,D,NaN,Alameda County Office of Education,Alameda,NaN,Y,NaN,EL,8,NaN,NaN,2.0,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018
21,1100170000000,D,NaN,Alameda County Office of Education,Alameda,NaN,Y,NaN,ELO,8,NaN,NaN,2.0,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148816,58727690000000,D,NaN,Wheatland Union High,Yuba,NaN,NaN,NaN,PI,1,NaN,NaN,0.0,NaN,NaN,NaN,0,0,0,0,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018
148817,58727690000000,D,NaN,Wheatland Union High,Yuba,NaN,NaN,NaN,RFP,2,NaN,NaN,3.0,NaN,NaN,NaN,0,0,0,0,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018
148818,58727690000000,D,NaN,Wheatland Union High,Yuba,NaN,NaN,NaN,SED,69,NaN,-28.4,55.0,NaN,-7.9,-20.6,2,1,2,160,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018
148819,58727690000000,D,NaN,Wheatland Union High,Yuba,NaN,NaN,NaN,SWD,18,18.0,-150.3,9.0,9.0,NaN,NaN,1,0,0,0,Y,-2.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018


## Math achievement indicator data
find the codebook [here](https://www.cde.ca.gov/ta/ac/cm/math18.asp)

In [ ]:
math = pandas.read_csv('lcff_data/mathdownload2018.csv')
math.shape

In [ ]:
math

In [ ]:
math = math.loc[math['rtype'] == 'D']
# select district level performance

# LCFF funding data

In [414]:
lcff = pandas.read_csv('lcff_data/lcffsnapshot18an.csv')
lcff.shape

(2243, 26)

In [415]:
lcff.columns
# leading AND trailing spaces!!!! inconsistently!!! 

Index([' County Code ', 'District Code', 'School Code',
       'Local Educational Agency ', 'Charter Number',
       ' TK/K-3 ADA\nTarget, B-5/B-1 ', ' 4 - 6 ADA\nTarget, B-6/B-2 ',
       ' 7 - 8 ADA\nTarget, B-7/B-3 ', ' 9 - 12 ADA\nTarget, B-8/B-4 ',
       'Unduplicated Pupil Percentage\nTarget, D-1',
       ' Base Grant Funding\nTarget, C-5 ',
       ' Supplemental Grant Funding\nTarget, D-7 ',
       ' Concentration Grant Funding\nTarget, E-8 ',
       ' Necessary Small Schools Allowance\nTarget, F-1 ',
       ' Add-On (Based on 2012-13 Targeted Instructional Improvement Block Grant)\nTarget, F-2 ',
       ' Add-On (Based on 2012-13 Home-to-School Transportation)\nTarget, F-3 ',
       ' Add-On (Based on 2012-13 Small School District Bus Replacement Program)\nTarget, F-4 ',
       ' Total LCFF Target Entitlement\nTarget, G-1/F-1 ',
       ' Total LCFF Floor Entitlement\nTransition, B-13 ',
       ' Current Year Gap Funding (100%)\nTransition, C-3 ',
       ' Economic Recovery Tar

In [416]:
lcff.columns = lcff.columns.str.strip()

In [417]:
# renaming important columns
lcff.rename(columns={'Charter Number': 'charter_number', 'Unduplicated Pupil Percentage\nTarget, D-1': 'unduplicated_pupil_percentage', 'Base Grant Funding\nTarget, C-5': 'base_grant', 'Supplemental Grant Funding\nTarget, D-7': 'supplemental_grant', 'Concentration Grant Funding\nTarget, E-8': 'concentration_grant', 'Total LCFF Target Entitlement\nTarget, G-1/F-1': 'total_main_grants'})

,County Code,District Code,School Code,Local Educational Agency,charter_number,"TK/K-3 ADA\nTarget, B-5/B-1","4 - 6 ADA\nTarget, B-6/B-2","7 - 8 ADA\nTarget, B-7/B-3","9 - 12 ADA\nTarget, B-8/B-4",unduplicated_pupil_percentage,base_grant,supplemental_grant,concentration_grant,"Necessary Small Schools Allowance\nTarget, F-1","Add-On (Based on 2012-13 Targeted Instructional Improvement Block Grant)\nTarget, F-2","Add-On (Based on 2012-13 Home-to-School Transportation)\nTarget, F-3","Add-On (Based on 2012-13 Small School District Bus Replacement Program)\nTarget, F-4",total_main_grants,"Total LCFF Floor Entitlement\nTransition, B-13","Current Year Gap Funding (100%)\nTransition, C-3","Economic Recovery Target\nTransition, D-1","Miscellaneous Adjustments\nTransition, E-1","Total Local Revenue or In-Lieu of Property Taxes\nTransition, F-2/F-7","Education Protection Account Entitlement\nTransition, F-5/F-9","Net State Aid\nTransition, F-6/F-10","Additional SA for MSA Guarantee\nTransition, H-1/H-2"
0,01,10017.0,112607.0,Envision Academy for Arts & Technology,811.0,-,-,-,359.23,0.7785,"$3,329,703","$518,435","$362,605",N/A,N/A,N/A,N/A,"$4,210,743","$3,871,076","$339,667",$-,$-,"$894,989","$683,070","$2,632,684",$-
1,01,10017.0,123968.0,Community School for Creative Education,1284.0,125.80,66.49,32.89,-,0.7936,"1,795,769","285,024","195,559",N/A,N/A,N/A,N/A,"2,276,352","2,124,388","151,964",-,-,"561,016","354,307","1,361,029",-
2,01,10017.0,124172.0,Yu Ming Charter,1296.0,252.38,134.11,48.83,-,0.1892,"3,474,375","131,471",-,N/A,N/A,N/A,N/A,"3,605,846","3,388,539","217,307",-,-,-,"683,750","2,922,096",-
3,01,10017.0,125567.0,Urban Montessori Charter,1383.0,271.70,101.50,26.45,-,0.3756,"3,212,111","241,293",-,N/A,N/A,N/A,N/A,"3,453,404","3,238,948","214,456",-,-,"995,692","627,724","1,829,988",-
4,01,10017.0,131581.0,Oakland Unity Middle School,1707.0,-,61.99,109.29,-,0.9199,"1,321,351","243,103","143,896",N/A,N/A,N/A,N/A,"1,708,350","1,620,979","87,371",-,-,"426,729","34,256","1,247,365",-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2238,58,72744.0,0.0,Plumas Lake Elementary,NaN,615.24,400.21,277.70,-,0.3868,"10,261,440","793,825",-,-,"4,433","49,203",-,"11,108,901","10,433,605","675,296","355,998",-,"1,029,589","2,348,862","8,086,448",-
2239,58,72751.0,0.0,Wheatland,NaN,580.88,389.53,284.05,-,0.4783,"9,947,133","951,543",-,-,"97,971","237,702",-,"11,234,349","10,546,226","688,123","309,994",-,"1,007,306","1,944,794","8,592,243",-
2240,58,72751.0,6118806.0,Wheatland Charter Academy,370.0,64.96,29.33,-,-,0.5069,"757,003","76,745",-,N/A,N/A,N/A,N/A,"833,748","793,112","40,636",-,-,"75,713","148,563","609,472",-
2241,58,72769.0,0.0,Wheatland Union High,NaN,-,-,-,761.94,0.4283,"7,062,422","604,967",-,-,-,"185,239",-,"7,852,628","7,260,075","592,553",-,-,"2,191,152","1,422,027","4,239,449",-


In [418]:
lcff_state_totals = lcff.tail(1)
lcff.drop(lcff.tail(1).index,inplace=True)
# extracting state totals, removing from df

In [419]:
# Slicing by district level cds code
lcff['County Code'] = lcff['County Code'].astype(str)
lcff['District Code'] = lcff['District Code'].astype(int)
lcff['District Code'] = lcff['District Code'].astype(str)
lcff['School Code'] = lcff['School Code'].astype(int)
lcff['School Code'] = lcff['School Code'].astype(str)
# Changing to strings
lcff = lcff.loc[lcff['School Code'] == '0']
# District level entries do not have a school level cds code
lcff['cds'] = lcff['County Code'] + lcff['District Code'] + lcff['School Code']
# Combining into cds code
col_name='cds'
first_col = lcff.pop(col_name)
lcff.insert(0, col_name, first_col)
# Moving to front
lcff.drop('County Code', axis=1, inplace=True)
lcff.drop('District Code', axis=1, inplace=True)
lcff.drop('School Code', axis=1, inplace=True)
# Dropping split cds columns
#lcff['cds'] = lcff['cds'].str.rstrip(lcff['cds'].iloc[-1])
# Removing school level code
#lcff = lcff[(lcff.cds.str.len() == 7)]
# Selecting 7 digit (district level) cds 
len(lcff)

944

In [427]:
lcff['cds'].str.len() == 7

10      False
16      False
17      False
19      False
20      False
        ...  
2233    False
2235    False
2238    False
2239    False
2241    False
Name: cds, Length: 944, dtype: bool

In [225]:
lcfft = set(lcff['cds'])
lcfft

{'0161192137646',
 '01611190',
 '1410140117994',
 '0910090930123',
 '19644850',
 '1062166133942',
 '47703180',
 '1262679111708',
 '37680230',
 '4369450113662',
 '10622570',
 '48705816116255',
 '50710920',
 '50105045030234',
 '24658210',
 '3776471119271',
 '4310439119024',
 '0161259115238',
 '52716210',
 '18641700',
 '4710470137372',
 '1910199132605',
 '3768106111195',
 '36677100',
 '37681220',
 '4110413135269',
 '37682960',
 '19647336119531',
 '37681300',
 '1964634119552',
 '4970912128074',
 '50757390',
 '19650780',
 '43696176048045',
 '34674396033799',
 '19647336016935',
 '35674700',
 '41690216044754',
 '43696740',
 '21654176113229',
 '34673550',
 '20102072030229',
 '1964733122606',
 '43693930',
 '30666960',
 '37683956040505',
 '41689240',
 '1910199135582',
 '52105206119606',
 '1964733100743',
 '3768338135913',
 '27754730',
 '4970938120121',
 '4870581115469',
 '30736430',
 '0761796132118',
 '1262687107110',
 '40687910',
 '20652010',
 '30666216094874',
 '19642950',
 '39754996118665',
 

In [233]:
elat.intersection(lcfft)

set()

In [ ]:
eladistrict_names.astype(str)

In [ ]:
eladistrict_names.sort()
# need to slice achievement data by rtype=d for this to work

In [ ]:
eladistrict_names
# ABC Unified at the top means working

In [ ]:
mathdistrict_names = math['districtname'].unique()

In [ ]:
mathdistrict_names.astype(str)

In [ ]:
mathdistrict_names.sort()

In [ ]:
mathdistrict_names

In [ ]:
s1 = set([1,2,3,4])

In [ ]:
s2 = set([3,4,5,6])

In [ ]:
s1.intersection(s2)
#3 and 4 are elements of both sets
#which elements from s1 are found in s2

In [ ]:
s1.difference(s2)

In [ ]:
elaset = set(eladistrict_names)

In [ ]:
mathset = set(mathdistrict_names)

In [ ]:
elaset.difference(mathset)
#disctricts missing from mathset

In [ ]:
mathset.difference(elaset)
#districts missing from elaset (none)

In [ ]:
adistrictset = set(adistrict_names)

In [ ]:
len(elaset.difference(adistrictset))
#need to add 'school district'
#copy performance data into new list

In [ ]:
len(adistrictset.difference(elaset))

In [ ]:
performance1 = set([word+" School District" for word in elaset])

In [ ]:
performance1.difference(adistrictset)

In [ ]:
s1 = 'Tulare elementary school'

In [ ]:
s1.replace(' elementary', '')

In [ ]:
performance1 = set([word+" School District" for word in elaset])

In [ ]:
adistrict2 = set([word.replace(' Elementary', '') for word in adistrictset])

In [ ]:
len(performance1.difference(adistrict2))

In [ ]:
adistrictset.difference(performance1)
# save into lists
# elementary/secondary/unified

In [ ]:
performance1

In [ ]:
adistrictset

In [ ]:
elaunique[0:10]

In [ ]:
elaunique = elaunique.astype(str)

In [ ]:
elaunique.sort()

In [ ]:
elaunique

In [ ]:
edistricts = uni.append([ele])
#unified/elementary districts

In [ ]:
edistricts.plot(column='origin', 
                legend=True, 
                cmap='winter', 
                figsize=(5,5)
               )

#subplots


In [ ]:
sdistricts = uni.append([seco])
#unified/secondary districts

In [ ]:
sdistricts.plot(column='origin',
                legend=True,
                cmap='winter', 
                figsize=(5,5)
               )

cds                                          1100170112607
rtype                                                    S
schoolname          Envision Academy for Arts & Technology
districtname            Alameda County Office of Education
countyname                                         Alameda
charter_flag                                             Y
coe_flag                                               NaN
dass_flag                                              NaN
studentgroup                                            EO
currdenom                                               35
currdenom_swd                                          NaN
currstatus                                           -13.5
priordenom                                              45
priordenom_swd                                         NaN
priorstatus                                           -3.8
change                                                -9.7
statuslevel                                             

In [ ]:
funding_lookup = lcff['School Code'].dropna()

In [ ]:
funding_lookup = funding_lookup.astype('int64')

In [ ]:
lcff_less_charter = lcff[lcff['Charter Number'].isnull()]
lcff_less_charter

inherent problems in matches - accept as structural problems in data gathering and storage (opacity)

Student group columns are shared by both performance indicators, can use to investigate performance by individual groups/impact by lcff funding. Create dictionaries using student groups and performance data?

In [ ]:
perf = pandas.Series(math.currstatus.values,index=math.studentgroup).to_dict()
perf

# Free and reduced meal plan data
Used by policy makers as a stand-in for a  segregation indicator.



In [64]:
frpm = pandas.read_csv('lcff_data/frpm1819.csv', encoding = "ISO-8859-1", skiprows=1)
frpm.shape

(10520, 28)

In [65]:
# renaming important columns
frpm = frpm.rename(columns={'Percent (%) \nEligible FRPM \n(K-12)': 'percent_eligible_k12_frpm', 'Percent (%) \nEligible Free \n(K-12)': 'percent_eligible_k12_free'})

In [68]:
frpm

,Academic Year,County Code,District Code,School Code,County Name,District Name,School Name,District Type,School Type,Educational \nOption Type,NSLP \nProvision \nStatus,Charter \nSchool \n(Y/N),Charter \nSchool \nNumber,Charter \nFunding \nType,IRC,Low Grade,High Grade,Enrollment \n(K-12),Free Meal \nCount \n(K-12),percent_eligible_k12_free,FRPM Count \n(K-12),percent_eligible_k12_frpm,Enrollment \n(Ages 5-17),Free Meal \nCount \n(Ages 5-17),Percent (%) \nEligible Free \n(Ages 5-17),FRPM Count \n(Ages 5-17),Percent (%) \nEligible FRPM \n(Ages 5-17),CALPADS Fall 1 \nCertification Status
0,2018-2019,1,10017,112607,Alameda,Alameda County Office of Education,Envision Academy for Arts & Technology,County Office of Education (COE),High Schools (Public),Traditional,NaN,Yes,0811,Directly funded,Y,9,12,385,262,68.1%,306,79.5%,370,249,67.3%,291,78.6%,Y
1,2018-2019,1,10017,123968,Alameda,Alameda County Office of Education,Community School for Creative Education,County Office of Education (COE),Elementary Schools (Public),Traditional,NaN,Yes,1284,Directly funded,Y,K,8,241,118,49.0%,120,49.8%,224,113,50.4%,115,51.3%,Y
2,2018-2019,1,10017,124172,Alameda,Alameda County Office of Education,Yu Ming Charter,County Office of Education (COE),Elementary Schools (Public),Traditional,NaN,Yes,1296,Directly funded,Y,K,8,445,58,13.0%,64,14.4%,445,58,13.0%,64,14.4%,Y
3,2018-2019,1,10017,125567,Alameda,Alameda County Office of Education,Urban Montessori Charter,County Office of Education (COE),Elementary Schools (Public),Traditional,NaN,Yes,1383,Directly funded,Y,K,8,432,113,26.2%,131,30.3%,418,111,26.6%,129,30.9%,Y
4,2018-2019,1,10017,130401,Alameda,Alameda County Office of Education,Alameda County Juvenile Hall/Court,County Office of Education (COE),Juvenile Court Schools,Juvenile Court School,NaN,No,NaN,NaN,N,K,12,53,53,100.0%,53,100.0%,50,50,100.0%,50,100.0%,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10515,2018-2019,58,72751,6056832,Yuba,Wheatland,Lone Tree Elementary,Elementary School District,Elementary Schools (Public),Traditional,NaN,No,NaN,NaN,N,K,5,366,77,21.0%,144,39.3%,357,76,21.3%,141,39.5%,Y
10516,2018-2019,58,72751,6056840,Yuba,Wheatland,Wheatland Elementary,Elementary School District,Elementary Schools (Public),Traditional,NaN,No,NaN,NaN,N,K,3,321,151,47.0%,177,55.1%,306,144,47.1%,169,55.2%,Y
10517,2018-2019,58,72751,6118806,Yuba,Wheatland,Wheatland Charter Academy,Elementary School District,Elementary Schools (Public),Traditional,NaN,Yes,0370,Locally funded,N,K,5,98,12,12.2%,43,43.9%,98,12,12.2%,43,43.9%,Y
10518,2018-2019,58,72769,1,Yuba,Wheatland Union High,"Nonpublic, Nonsectarian Schools",High School District,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0,0.0%,0,0.0%,2,0,0.0%,0,0.0%,Y


In [ ]:
frpm['School Type'].unique()
# is there any reason to cut any of these out?

In [ ]:
frpm.head()
# lookup table

In [ ]:
# frpm groupby district 
# 1) remove %
# 2) convert to float
# 3) groupby 
# 4) compare with dissimilarity index to validate proxy <--- YOU ARE HERE

In [ ]:
frpm['percent_eligible_k12_free'] = frpm['percent_eligible_k12_free'].str.strip('%')
frpm['percent_eligible_k12_free'].astype(float)

In [ ]:
frpm['percent_eligible_k12_frpm'] = frpm['percent_eligible_k12_frpm'].str.strip('%')
frpm['percent_eligible_k12_frpm'].astype(float)

In [ ]:
frpm.head()

In [ ]:
funding_lookup

In [ ]:
frmp['District Code']

In [ ]:
good = [code for code in funding_lookup if code in frmp['District Code']]

In [ ]:
good[-10:]

In [ ]:
for code in funding_lookup:
    if code in frmp['District Code']:
        print(code)

In [ ]:
frmp['Percent (%) \nEligible FRPM \n(K-12)']
# these are by school only.. will need to do an average to get a figure to represent the county, or do something else?

# Census Data

#### SCAG counties and their FIPS codes:
<br> Los Angeles (06037)<br> Orange (06059)<br> Riverside (06065)<br> San Bernardino (06071)<br> San Diego (06073)<br>Imperial (06025)<br> Ventura (06111)

In [ ]:
cacom = Community.from_census(state_fips='06')

In [ ]:
scag = Community.from_census(county_fips=['06025', '06037', '06059', '06071', '06111', '06073', '06065'])

In [ ]:
rivco = Community.from_census(county_fips=['06065'])

In [ ]:
orangeco = Community.from_census(county_fips=['06059'])

In [ ]:
laco = Community.from_census(county_fips=['06037'])

In [ ]:
sbco = Community.from_census(county_fips=['06071'])

In [ ]:
sdco = Community.from_census(county_fips=['06073'])

In [ ]:
f, ax = plt.subplots(1, figsize=(30, 30))
ax = rivco.gdf[rivco.gdf.year==2010].plot('p_poverty_rate_over_65', axes=ax)
plt.show()

# change gdf, variable, and year

### This 'plot' function will produce three maps, one from each of the census data years. We can use it to pass in communities and a specific variable to provide a preliminary view of changes over 30 years.

In [ ]:
def plot(community, column):
    fig, axs = plt.subplots(1,3, figsize=(20,20))
    axs=axs.flatten()
    
    community.gdf[community.gdf.year==1990].dropna(subset=[column]).plot(column=column, scheme='quantiles', cmap='Greens', k=7, ax=axs[0])
    axs[0].axis('off')
    axs[0].set_title('1990')

    community.gdf[community.gdf.year==2000].dropna(subset=[column]).plot(column=column, scheme='quantiles', cmap='Greens', k=7, ax=axs[1])
    axs[1].axis('off')
    axs[1].set_title('2000')

    community.gdf[community.gdf.year==2010].dropna(subset=[column]).plot(column=column, scheme='quantiles', cmap='Greens', k=7, ax=axs[2])
    axs[2].axis('off')
    axs[2].set_title('2010')

In [ ]:
datasets.codebook()

### Variables of interest from census:
#### economic indicators
median_household_income <br>
median_home_value OR median_contract_rent<br>
median_income_whitehh<br>
median_income_blackhh<br>
median_income_hispanichh<br>
n_edu_college_greater - persons with at least 4-year degree (also available as %)<br>
n_edu_hs_less - persons with less than high school (also available as %)<br>

#### demographic indicators
n_recent_immigrant_pop (within the last 10 years)<br>
n_limited_english OR n_other_language<br>
n_hispanic_persons<br>
n_black_persons<br>
n_white_persons

In [ ]:
from segregation.aspatial import GiniSeg
index = GiniSeg(scag.gdf, 'n_mexican_pop', 'n_total_pop')
type(index)

# MERGE SCHOOL DISTRICTS WITH CENSUS DATA

## Alternative community construction

Can also construct communities from longitudinal tract database (ltdb)

In [ ]:
datasets.msas()[datasets.msas().name.str.startswith('Riverside')]

In [ ]:
datasets.msas()[datasets.msas().name.str.startswith('Orange')]

In [ ]:
datasets.msas()[datasets.msas().name.str.startswith('San')]

In [ ]:
LA = Community.from_ltdb(msa_fips='31080')

In [ ]:
SD = Community.from_ltdb(msa_fips='41740')

In [ ]:
riverside = Community.from_ltdb(msa_fips='40140')